In [2]:
import numpy as np
import torch
import os
import h5py
import pickle
import fnmatch
import cv2
import sys
from time import time
from torch.utils.data import TensorDataset, DataLoader

In [3]:
act_path = "/home/zfei/code/act-plus-plus/"
sys.path.append(act_path)
from utils import EpisodicDataset
from utils import find_all_hdf5, flatten_list, get_norm_stats

%load_ext autoreload
%autoreload 2

In [4]:
dataset_path_list = find_all_hdf5("/mnt/data1/act/train_act_scara_3cam/data1/", skip_mirrored_data=False)
norm_stats, all_episode_len = get_norm_stats(dataset_path_list)

camera_names = ['wrist', 'wrist_down', 'top']
train_episode_ids = [0, 1]
train_episode_len = [all_episode_len[i] for i in train_episode_ids]

chunk_size = 100
policy_class = "EPACT"

Found 51 hdf5 files


In [6]:
train_dataset = EpisodicDataset(dataset_path_list, camera_names, norm_stats, train_episode_ids, train_episode_len, chunk_size, policy_class)


augment_images:  False
Initializing transformations


In [8]:
index = 100
image_data, qpos_data, action_data, is_pad, end_pose_data = train_dataset[0]
print("image_data: ", image_data.shape)
print("qpos_data: ", qpos_data.shape)
print("action_data: ", action_data.shape)
print("end_pose_data: ", end_pose_data.shape)

image_data:  torch.Size([3, 3, 480, 640])
qpos_data:  torch.Size([4])
action_data:  torch.Size([100, 5])
end_pose_data:  torch.Size([100, 5])
